In [457]:
import numpy as np
import copy
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder , OneHotEncoder
from transformers import BertTokenizer, BertModel
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.neural_network import MLPClassifier

Data processing

Run this 1st

In [452]:
df = pd.read_csv('test.csv')
X = df['text']
Y = df['category']

label_encoder = LabelEncoder()
Y_encoded = label_encoder.fit_transform(Y)


onehot_encoder = OneHotEncoder()
Y = onehot_encoder.fit_transform(Y_encoded.reshape(-1, 1))
X= np.array(X)
Y = Y.toarray()
print(X.shape)
print(Y[:20])


(500,)
[[0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]]


In [376]:


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

Making bert embeddings

In [362]:
def get_bert_embeddings(texts, tokenizer, model, max_len=512):
    model.eval()
    embeddings = []
    with torch.no_grad():
        for text in texts:
            encoding = tokenizer.encode_plus(
                text,
                add_special_tokens=True,
                max_length=max_len,
                padding='max_length',
                return_attention_mask=True,
                return_tensors='pt'
            )
            input_ids = encoding['input_ids']
            attention_mask = encoding['attention_mask']
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            pooled_output = outputs[1]
            embeddings.append(pooled_output)
    embeddings = torch.cat(embeddings)
    return embeddings.numpy()

This following cell creates bert embedding for your text. this might take a long time

In [389]:
'''Run this 2nd'''
X = [text[:512] for text in X] #because bert tokens have limil of 512
X_embeddings = get_bert_embeddings(X, tokenizer, model,max_len=512)
X_train, X_test, Y_train, Y_test = train_test_split(X_embeddings, Y, test_size=0.2, random_state=42)


Save the embedding

In [390]:
import pickle  
with open('embeddings_file.npy', 'wb') as f:
    pickle.dump(X_embeddings, f)

## Training The model 

In [391]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [453]:
Mymodel = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.4),  # Optional: regularization with dropout
    Dense(64, activation='relu'),
    Dropout(0.2),  # Optional: regularization with dropout
    Dense(5, activation='softmax')  # Output layer for binary classification
])

C:\Users\Asjad1\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [454]:
Mymodel.compile(optimizer='adam',
              loss='binary_crossentropy',  # For binary classification
              metrics=['accuracy'])


In [455]:
Y_train = Y_train.toarray()

Y_test =  Y_test.toarray()

AttributeError: 'numpy.ndarray' object has no attribute 'toarray'

In [456]:

history = Mymodel.fit(X_train, Y_train,
                    epochs=200,  # Adjust number of epochs as needed
                    batch_size=32,  # Adjust batch size as needed
                    validation_data=(X_test, Y_test))



Epoch 1/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 33ms/step - accuracy: 0.1905 - loss: 0.5926 - val_accuracy: 0.3000 - val_loss: 0.4970
Epoch 2/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2884 - loss: 0.5278 - val_accuracy: 0.3800 - val_loss: 0.4723
Epoch 3/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3192 - loss: 0.5100 - val_accuracy: 0.4100 - val_loss: 0.4692
Epoch 4/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3368 - loss: 0.4895 - val_accuracy: 0.3900 - val_loss: 0.4678
Epoch 5/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.3577 - loss: 0.4821 - val_accuracy: 0.5400 - val_loss: 0.4556
Epoch 6/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4389 - loss: 0.4510 - val_accuracy: 0.5100 - val_loss: 0.4403
Epoch 7/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4060 - loss: 0.4505 - val_accuracy: 0.4500 - val_loss: 0.4342
Epoch 8/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.4338 - loss: 0.4537 - val_accuracy: 0.

plot the learning curve

In [458]:
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, solver='adam', random_state=42)
mlp.fit(X_train, Y_train)


c:\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(max_iter=300, random_state=42)

ValueError: dtype='numeric' is not compatible with arrays of bytes/strings.Convert your data to numeric values explicitly instead.

In [428]:
Mymodel.summary()

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_30 (Dense)                │ (None, 128)            │        98,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_31 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_32 (Dense)                │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321,041 (1.22 MB)

 Trainable params: 107,013 (418.02 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 214,028 (836.05 KB)

Making predictions on new examples


In [429]:
def get_bert_embedding_for_text(text, tokenizer, model, max_len=512):
    model.eval()
    with torch.no_grad():
        encoding = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_len,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt'
        )
        input_ids = encoding['input_ids']
        attention_mask = encoding['attention_mask']
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]
    return pooled_output.numpy().flatten()

In [466]:
new_text = "berlin cheers for anti-nazi film a german movie about an anti-nazi resistance heroine has drawn loud applause at berlin film festival.  sophie scholl - the final days portrays the final days of the member of the white rose movement. scholl  21  was arrested and beheaded with her brother  hans  in 1943 for distributing leaflets condemning the  abhorrent tyranny  of adolf hitler. director marc rothemund said:  i have a feeling of responsibility to keep the legacy of the scholls going.   we must somehow keep their ideas alive   he added.  the film drew on transcripts of gestapo interrogations and scholl s trial preserved in the archive of communist east germany s secret police. their discovery was the inspiration behind the film for rothemund  who worked closely with surviving relatives  including one of scholl s sisters  to ensure historical accuracy on the film. scholl and other members of the white rose resistance group first started distributing anti-nazi leaflets in the summer of 1942. they were arrested as they dropped leaflets at munich university calling for a  day of reckoning  with adolf hitler s regime. the film focuses on the six days from scholl s arrest to the intense trial which saw scholl initially deny the charges and ended with a defiant appearance. it is one of three german films vying for a top prize at the festival.  a south african film version of bizet s tragic opera carmen shot in cape town in the xhosa language has also premiered at the berlin festival. the film is entitled u-carmen ekhayelitsha or carmen in khayelitsha after the township in which the story is set. it is performed by a 40-strong music and theatre troupe in their debut film performance. the film is the first south african feature in 25 years and only the second to be nominated for a golden bear award."


# Get the BERT embedding for the new text
new_text_embedding = get_bert_embedding_for_text(new_text, tokenizer, model)

# Reshape the embedding to match the input shape expected by the classifier
new_text_embedding = new_text_embedding.reshape(1, -1)

# Make a prediction using the trained classifier
prediction = Mymodel.predict(new_text_embedding)
prediction = prediction.reshape(5)
print(prediction)
predicted_label = label_encoder.inverse_transform(prediction)

print(predicted_label)



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
[9.7745823e-10 9.9999952e-01 3.7061182e-07 1.4679307e-07 2.3093636e-09]


ValueError: y contains previously unseen labels: [9.7745823e-10 2.3093636e-09 1.4679307e-07 3.7061182e-07 9.9999952e-01]